In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(plyr)
library(data.table)
library(parallel)
library(purrr)
library(rstatix)
source('/home//jupyter/BRI_Figures/helper_function/helper_function_IHA.r')

------------------------------------------------------------------------------

You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)

------------------------------------------------------------------------------


Attaching package: ‘plyr’


The following objects are masked from ‘package:dplyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last



Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose


The following object is masked from ‘package:plyr’:

    compact



Attaching package: ‘rstatix’


The following objects are masked from ‘package:plyr’:

    desc, mutate


The following object is masked from ‘package:stats’:

    filter




# Read meta data

In [2]:
meta_data<-read.csv('/home//jupyter/BRI_Figures_Final_V2/Figure5/01_Frequency_Comparison/selected_samples_with_acutal_flu_year.csv')

# Read Freq Files and do subsetting

In [3]:
df_freq<-read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/diha_AIFI_L3_frequencies_2024-05-05.csv")

In [4]:
df_freq_Y1<-df_freq %>% filter(specimen.specimenGuid %in% meta_data$pbmc_sample_id)

In [5]:
df_freq_Y1<-left_join(df_freq_Y1,meta_data[c('pbmc_sample_id','Flu_Year','Flu_Day')],by=c("specimen.specimenGuid"="pbmc_sample_id"))

# Read Freq of CD27- subtypes

In [6]:
df_subtype<-read.csv('CD27neg_effector_B_subtype.csv')

In [7]:
df_subtype$AIFI_L3<-paste0("CD27- effector B-",df_subtype$switched_status)

In [8]:
df_subtype$AIFI_L3_count<-df_subtype$count

# Overlaps

In [9]:
df_freq_Y1<-df_freq_Y1 %>% filter(Flu_Day =='Day 0')

In [10]:
n_distinct(df_freq_Y1$sample.sampleKitGuid)

[1] 136

In [11]:
df_freq_Y1<-df_freq_Y1 %>% filter(!AIFI_L3=="CD27- effector B cell")

In [12]:
df<-rbind.fill(df_freq_Y1,df_subtype)

In [13]:
df<-df %>% filter(AIFI_L3%in% c('CD27- effector B-nonswitched',
                            'CD27- effector B-switched',
                            'CD27- effector B-undetermined',
                            'Activated memory B cell',
                            'CD95 memory B cell',
                            'Core memory B cell',
                            'Early memory B cell',
                       'CD27+ effector B cell',
                            'Type 2 polarized memory B cell'))

In [14]:
df <- df %>% 
dplyr::group_by(specimen.specimenGuid)%>% 
dplyr::mutate(sum_counts = sum(AIFI_L3_count))%>% 
dplyr::mutate(frequency = AIFI_L3_count/sum_counts*100)%>% 
ungroup()%>% dplyr::group_by(specimen.specimenGuid)%>% 
dplyr::mutate(frequency_clr = clr_transform(frequency)) %>% ungroup()


In [15]:
df<-left_join(df ,meta_data[c("sample.sampleKitGuid","Flu_Year")],by=c("sample.sampleKitGuid"))

In [19]:
write.csv(df,'memory_b_freq.csv')

In [17]:
df %>% dplyr::group_by(Flu_Year.y,AIFI_L3)%>% wilcox_test(frequency_clr~cohort.cohortGuid)%>% filter(AIFI_L3%in% c('CD27- effector B-nonswitched',
                            'CD27- effector B-switched',
                            'CD27- effector B-undetermined'))

AIFI_L3,Flu_Year.y,.y.,group1,group2,n1,n2,statistic,p
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>
CD27- effector B-nonswitched,2020-2021,frequency_clr,BR1,BR2,26,42,357,0.01660
CD27- effector B-switched,2020-2021,frequency_clr,BR1,BR2,26,42,517,0.72100
CD27- effector B-undetermined,2020-2021,frequency_clr,BR1,BR2,4,4,9,0.88600
CD27- effector B-nonswitched,2021-2022,frequency_clr,BR1,BR2,26,42,327,0.00527
CD27- effector B-switched,2021-2022,frequency_clr,BR1,BR2,26,39,492,0.84700
CD27- effector B-undetermined,2021-2022,frequency_clr,BR1,BR2,3,2,4,0.80000


In [ ]:
df